In [2]:
%load_ext autoreload
%autoreload 2
import risktools as rt
from pandas_datareader import data
import json
import os
import pandas as pd

{'tradeDate': ['2023-08-18'], 'settleDate': ['2023-08-20'], 'dt': [0.0833], 'interpWhat': ['discount'], 'interpHow': ['spline']}


In [3]:
def _load_json(fn, dataframe=True):
    # path = os.path.dirname(__file__)
    fp = os.path.join(fn)
    with open(fp) as js:
        if dataframe == True:
            df = pd.read_json(js)
            df.columns = df.columns.str.replace(".", "_")
        else:
            df = json.load(js)

    return df

In [23]:
pd.read_json('dfwide.json').set_index('date').sort_index()

,CL01,CL02,CL03,CL04,CL05,CL06,CL07,CL08,CL09,CL10,...,LSW03,LSW04,LSW05,LSW06,LSW07,LSW08,LSW09,LSW10,LSW11,LSW12
date,,,,,,,,,,,,,,,,,,,,,
2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-05,33.78,33.51,33.01,32.41,31.82,31.26,30.76,30.32,29.94,29.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-06,33.70,33.48,33.04,32.45,31.88,31.32,30.82,30.38,30.00,29.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-07,33.62,33.40,32.97,32.42,31.85,31.28,30.78,30.34,29.97,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-08,33.98,33.73,33.28,32.73,32.17,31.61,31.11,30.67,30.30,30.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27,89.08,87.83,86.43,85.06,83.84,82.77,81.84,81.00,80.27,79.61,...,-3.10,NaN,-2.85,-2.60,-2.50,-2.60,-2.70,-2.50,-3.05,-3.15
2022-10-28,87.90,86.61,85.17,83.83,82.62,81.57,80.66,79.85,79.14,78.49,...,-3.10,NaN,-2.85,-2.60,-2.50,-2.60,-2.70,-2.50,-3.05,-3.15
2022-10-31,86.53,85.40,84.10,82.88,81.78,80.83,80.00,79.25,78.57,77.95,...,-3.10,NaN,-2.85,-2.60,-2.60,-2.55,-2.50,-2.50,-2.90,-3.15


In [46]:
ac = _load_json("../pytest/data/promptBeta.json").round(4).drop("contract", axis=1)

dfwide = rt.data.open_data("dflong").unstack(0)
col_mask = dfwide.columns[dfwide.columns.str.contains("CL")]
dfwide = dfwide[col_mask]
dfwide = dfwide[~dfwide.index.isin(["2020-04-20", "2020-04-21"])]
dfwide = dfwide.loc['2004-01-02':'2022-12-15',:]
x = rt.returns(df=dfwide, ret_type="abs", period_return=1)
x = rt.roll_adjust(df=x, commodity_name="cmewti", roll_type="Last_Trade")

ts = (
    rt.prompt_beta(df=x, period="all", beta_type="all", output="betas")
    .round(4)
    .reset_index(drop=True)
)

ts.head()

,all,bull,bear
0,1.0000,1.0000,1.0000
1,0.9508,0.8923,0.9765
2,0.9223,0.8599,0.9585
3,0.8971,0.8334,0.9399
4,0.8746,0.8110,0.9230


In [31]:
ac.head()

,all,bull,bear
0,1.0000,1.0000,1.0000
1,0.9506,0.8923,0.9778
2,0.9210,0.8597,0.9574
3,0.8953,0.8327,0.9387
4,0.8724,0.8099,0.9213
